# <center>________________________________________________________________</center>

# <center>SQL QUERIES FOR SPACEX FALCON 9 LAUNCH DATA</center>

# <center>________________________________________________________________</center>

# Introduction
***

In this project, we will create some SQL queries to gather information from SpaceX Launch Data. The dataset includes a record for each payload carried with Falcon 9 rocket during a SpaceX mission into outer space.

# Libraries
***

In [ ]:
#!pip install pandas
#!pip install sqlalchemy
#!pip install sqlite3
#!pip install ipython-sql

In [1]:
import csv, sqlite3
import pandas as pd

# Data Acquisition and Database Connection
***

First we will load the SQL extension, import our data into a database, and establish a connection with the database:

In [2]:
%load_ext sql

In [3]:
con = sqlite3.connect("my_database.db")
cur = con.cursor()

In [4]:
%sql sqlite:///my_database.db

In [5]:
df = pd.read_csv("https://github.com/efeyemez/Portfolio/raw/main/Datasets/SpaceX_Falcon_9/launch_data.csv")
print(df.shape)
df.head()

(101, 10)


,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,06/04/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,12/08/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,10/08/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,03/01/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [6]:
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

101

# Queries
***

### Names of the unique launch sites  in the space mission:

In [7]:
%%sql

SELECT DISTINCT "Launch_Site" FROM SPACEXTBL;

 * sqlite:///my_database.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


### 5 records where launch sites begin with the string 'CCA':

In [8]:
%%sql

SELECT * FROM SPACEXTBL
    WHERE "Launch_Site" LIKE "CCA%" LIMIT 5;

 * sqlite:///my_database.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
06/04/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
12/08/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
10/08/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
03/01/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Total payload mass carried by boosters launched by NASA (CRS):

In [9]:
%%sql

SELECT SUM("PAYLOAD_MASS__KG_") AS "Total Payload Mass (kg) by NASA (CRS)" FROM SPACEXTBL
    WHERE "Customer" = "NASA (CRS)";

 * sqlite:///my_database.db
Done.


Total Payload Mass (kg) by NASA (CRS)
45596


### Average payload mass carried by booster version F9 v1.1:

In [10]:
%%sql

SELECT AVG("PAYLOAD_MASS__KG_") AS "Average Payload Mass (kg) by F9 v1.1" FROM SPACEXTBL
    WHERE "Booster_Version" = "F9 v1.1";

 * sqlite:///my_database.db
Done.


Average Payload Mass (kg) by F9 v1.1
2928.4


### Date when the first succesful landing on a ground pad was acheived:

In [11]:
%%sql

SELECT MIN(substr("Date",7,4) || substr("Date",4,2) || substr("Date",1,2)) as "Date (YYYYMMDD)" FROM SPACEXTBL
    WHERE "Landing_Outcome" = "Success (ground pad)";

 * sqlite:///my_database.db
Done.


Date (YYYYMMDD)
20151222


### Versions of the boosters that landed successfully on a drone ship with a payload between 4000 and 6000 kg:

In [12]:
%%sql

SELECT DISTINCT "Booster_Version" FROM SPACEXTBL
    WHERE "Landing_Outcome" = "Success (drone ship)" AND ("PAYLOAD_MASS__KG_" > 4000 AND "PAYLOAD_MASS__KG_" < 6000);

 * sqlite:///my_database.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


### Total number of successful and failed missions:

In [13]:
%%sql

SELECT DISTINCT "Mission_Outcome" AS "Mission Outcome", COUNT("Mission_Outcome") AS "Count" FROM SPACEXTBL
    GROUP BY "Mission_Outcome";

 * sqlite:///my_database.db
Done.


Mission Outcome,Count
Failure,1
Success,100


### Versions of the boosters that carried the maximum payload (with a subquery):

In [14]:
%%sql

SELECT "Booster_Version" FROM SPACEXTBL
    WHERE "PAYLOAD_MASS__KG_" = (SELECT MAX("PAYLOAD_MASS__KG_") FROM SPACEXTBL);

 * sqlite:///my_database.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


### Failed drone ship landings in 2015, with respective booster versions, launch sites, and the month of the year:

In [15]:
%%sql

SELECT substr("Date", 4, 2) AS "Month of the Year 2015", "Landing_Outcome", "Booster_Version", "Launch_Site" FROM SPACEXTBL
    WHERE "Landing_Outcome" = "Failure (drone ship)" AND substr(Date,7,4)='2015';

 * sqlite:///my_database.db
Done.


Month of the Year 2015,Landing_Outcome,Booster_Version,Launch_Site
10,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Successful landings between 4th of June, 2010 and 20th of March, 2017 in descending order:

In [16]:
%%sql

SELECT "Date", "Landing_Outcome" FROM SPACEXTBL
    WHERE substr("Landing_Outcome", 1, 7)="Success" AND
    (substr("Date",7,4) || substr("Date",4,2) || substr("Date",1,2)) BETWEEN "20100604" AND "20170320"
    ORDER BY (substr("Date",7,4) || substr("Date",4,2) || substr("Date",1,2));

 * sqlite:///my_database.db
Done.


Date,Landing_Outcome
22/12/2015,Success (ground pad)
27/05/2016,Success (drone ship)
05/06/2016,Success (drone ship)
18/07/2016,Success (ground pad)
04/08/2016,Success (drone ship)
14/08/2016,Success (drone ship)
05/01/2017,Success (ground pad)
14/01/2017,Success (drone ship)
19/02/2017,Success (ground pad)
06/03/2017,Success (ground pad)


# <center>________________________________________________________________</center>